In [15]:
!pip install transformers

In [16]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel, pipeline
from collections import defaultdict

In [17]:
with open('childhood_friends_chat.txt', encoding="utf-8") as file:
  total_chat = file.readlines()

In [18]:
people = defaultdict(list)
for l in total_chat:
  sen = l.rstrip("\n")
  if ']' in sen:
    name_and_msg = sen.split(']')[1]
    if ':' in name_and_msg:
      name_and_msg = name_and_msg.strip()
      name = name_and_msg.split(':')[0].replace(' ', '_')
      msg = name_and_msg.split(':')[1]
      people[name].append(msg)
min_msgs = np.min([len(msgs) for person, msgs in people.items()])
for person, msgs in people.items():
  while len(msgs) > min_msgs:
    msgs.pop()
df = pd.DataFrame(data=people)
df.head()

,Yaniv_Ofer,Gaby_Naiman,Tomer_Schmidt_Barad,Yaniv_Cohen,Dvir_Cohen,חברי_ילדות
0,"גבי,",בודק..,"גבי, אתה יותר איטי מויסטה",הוא ישן,ארון כהן לאייפונים,‎Messages and calls are end-to-end encrypted....


In [19]:
def get_positivity(string):
  states = sentiment_analysis(string)[0]
  for state in states:
    if state['label'] == 'positive':
      break
  return state['score']

In [20]:
def get_negativity(string):
  states = sentiment_analysis(string)[0]
  for state in states:
    if state['label'] == 'negative':
      break
  return state['score']

In [21]:
def get_neutrality(string):
  states = sentiment_analysis(string)[0]
  for state in states:
    if state['label'] == 'natural':
      break
  return state['score']

In [22]:
tokenizer = AutoTokenizer.from_pretrained("avichr/heBERT_sentiment_analysis")
model = AutoModel.from_pretrained("avichr/heBERT_sentiment_analysis")

sentiment_analysis = pipeline(
    "sentiment-analysis",
    model="avichr/heBERT_sentiment_analysis",
    tokenizer="avichr/heBERT_sentiment_analysis",
    return_all_scores = True
)

Some weights of the model checkpoint at avichr/heBERT_sentiment_analysis were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
for person in df.columns:
  df[f'{person}_positivity_index'] = df[person].apply(lambda r: get_positivity(r))
  df[f'{person}_negativity_index'] = df[person].apply(lambda r: get_negativity(r))
  df[f'{person}_neutrality_index'] = df[person].apply(lambda r: get_neutrality(r))

In [24]:
for person in people.keys():
  df[f'{person}_positivity_score'] = ((df[f'{person}_positivity_index'] + 
                                         df[f'{person}_negativity_index']) / 2 + 
                                        df[f'{person}_neutrality_index']) / 2

In [25]:
people_scores = []
for name in people.keys():
  score = round(df[f'{name}_positivity_score'].mean(), 5)
  people_scores.append((name, score))
people_scores.sort(reverse=True, key=lambda x:x[1])
for person in people_scores:
  print(f'{person[0]} positivity score: {person[1]}')

חברי_ילדות positivity score: 0.49248
Dvir_Cohen positivity score: 0.48377
Gaby_Naiman positivity score: 0.44758
Yaniv_Cohen positivity score: 0.25098
Yaniv_Ofer positivity score: 0.25011
Tomer_Schmidt_Barad positivity score: 0.25003
